<a href="https://colab.research.google.com/github/IqraAziz15/OpenRestaurantFYP/blob/master/model/sentiment_analysis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries for model
# %tensorflow_version 2.x
# import tensorflow as tens_flow
# import tensorflow_datasets as tens_flow_ds
# import tensorflow_hub as tens_hub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
#importing dataset from local system
from google.colab import files
uploaded = files.upload() 

Saving food_reviews.csv to food_reviews.csv
Saving Reviews.csv to Reviews.csv


In [ ]:
#read data from uploaded files
#dataset1
amazon_fine_food_reviews_dataset = pd.read_csv('Reviews.csv')
#dataset2
food_reviews_dataset = pd.read_csv('food_reviews.csv')

In [ ]:
#datasets info

In [ ]:
amazon_fine_food_reviews_dataset.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
food_reviews_dataset.head(5)

,Text,y,Source,Restaurant,Unnamed: 4
0,Thoroughly enjoyed food and place. Very good t...,1.0,Google,Roasters,NaN
1,The ambience of the place was good. The menu h...,1.0,Google,Roasters,NaN
2,Loved the food! The quality was top notch and ...,1.0,Google,Roasters,NaN
3,Great burgers by jessie's,1.0,Facebook,Jessies,NaN
4,sauce and mashed potatoes are very less in qua...,0.0,Facebook,Jessies,NaN


In [ ]:
amazon_fine_food_reviews_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219492 entries, 0 to 219491
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      219492 non-null  int64 
 1   ProductId               219492 non-null  object
 2   UserId                  219492 non-null  object
 3   ProfileName             219484 non-null  object
 4   HelpfulnessNumerator    219492 non-null  int64 
 5   HelpfulnessDenominator  219492 non-null  int64 
 6   Score                   219492 non-null  int64 
 7   Time                    219492 non-null  int64 
 8   Summary                 219483 non-null  object
 9   Text                    219492 non-null  object
dtypes: int64(5), object(5)
memory usage: 16.7+ MB


In [ ]:
food_reviews_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Text        61 non-null     object 
 1   y           61 non-null     float64
 2   Source      31 non-null     object 
 3   Restaurant  39 non-null     object 
 4   Unnamed: 4  0 non-null      float64
dtypes: float64(2), object(3)
memory usage: 5.4+ KB


In [ ]:
amazon_fine_food_reviews_dataset.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,219492.000000,219492.000000,219492.000000,219492.000000,2.194920e+05
mean,109886.224436,1.726938,2.215684,4.175441,1.295793e+09
std,63526.487224,7.440930,8.094666,1.310583,4.793169e+07
min,1.000000,0.000000,0.000000,1.000000,9.393408e+08
25%,54873.750000,0.000000,0.000000,4.000000,1.270166e+09
50%,109746.500000,0.000000,1.000000,5.000000,1.310515e+09
75%,164915.250000,2.000000,2.000000,5.000000,1.332461e+09
max,219976.000000,866.000000,923.000000,5.000000,1.351210e+09


In [ ]:
food_reviews_dataset.describe()

In [ ]:
#data preprocessing starts here

In [ ]:
#drop rows containing null values
amazon_fine_food_reviews_dataset.dropna()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
219487,219972,B000FDKQBK,A3B7MD6W6XUAOM,"G Y ""GY WY""",1,2,5,1257033600,Flax is great Omega3,Amazon is always great to order from. Hodgson...
219488,219973,B000FDKQBK,A2PA9Z65V5ME75,"M. Fatima Muhammad ""doggone skater 550""",0,1,4,1326067200,Good Health Search,I bought this item to assist my own health ove...
219489,219974,B000FDKQBK,A26XFG2AAD3UMD,jo,0,1,5,1305676800,Hodgson Mill Milled Flax Seed,Hodgson Mill Milled Flax Seed was received pro...
219490,219975,B000FDKQBK,A2QYA3WQYE2YLE,Sparkey,0,1,5,1292976000,Flax Seed is as advertised,Very satisfied with the product. I use a lot ...


In [ ]:
# amazon_fine_food_reviews_dataset = amazon_fine_food_reviews_dataset.iloc[:200000, :]

In [ ]:
amazon_fine_food_reviews_dataset

In [ ]:
# converting all test to lower dataset1
amazon_fine_food_reviews_dataset['Text'] = amazon_fine_food_reviews_dataset['Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
amazon_fine_food_reviews_dataset['Text'].head()

0    i have bought several of the vitality canned d...
1    product arrived labeled as jumbo salted peanut...
2    this is a confection that has been around a fe...
3    if you are looking for the secret ingredient i...
4    great taffy at a great price. there was a wide...
Name: Text, dtype: object

In [ ]:
# converting all test to lower dataset2
food_reviews_dataset['Text'] = food_reviews_dataset['Text'].astype(str)
food_reviews_dataset['Text'] = food_reviews_dataset['Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
food_reviews_dataset['Text'].head()

0    thoroughly enjoyed food and place. very good t...
1    the ambience of the place was good. the menu h...
2    loved the food! the quality was top notch and ...
3                            great burgers by jessie's
4    sauce and mashed potatoes are very less in qua...
Name: Text, dtype: object

In [ ]:
# removing punctuaton marks dataset1
amazon_fine_food_reviews_dataset['Text'] = amazon_fine_food_reviews_dataset['Text'].str.replace('[^\w\s]','')
amazon_fine_food_reviews_dataset['Text'].head()

0    i have bought several of the vitality canned d...
1    product arrived labeled as jumbo salted peanut...
2    this is a confection that has been around a fe...
3    if you are looking for the secret ingredient i...
4    great taffy at a great price there was a wide ...
Name: Text, dtype: object

In [ ]:
# removing punctuaton marks dataset2
food_reviews_dataset['Text'] = food_reviews_dataset['Text'].str.replace('[^\w\s]','')
food_reviews_dataset['Text'].head()

0    thoroughly enjoyed food and place very good ta...
1    the ambience of the place was good the menu ha...
2    loved the food the quality was top notch and t...
3                             great burgers by jessies
4    sauce and mashed potatoes are very less in qua...
Name: Text, dtype: object

In [ ]:
#Add y column

In [ ]:
amazon_fine_food_reviews_dataset['y']= amazon_fine_food_reviews_dataset.Score
amazon_fine_food_reviews_dataset.y[amazon_fine_food_reviews_dataset.Score <= 3] = 0
amazon_fine_food_reviews_dataset.y[amazon_fine_food_reviews_dataset.Score >  3] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#dataset2 already has y column

In [ ]:
amazon_fine_food_reviews_dataset.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,y
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,i have bought several of the vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,product arrived labeled as jumbo salted peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",this is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,if you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,great taffy at a great price there was a wide ...,1


In [ ]:
#merging both datasets

In [ ]:
#data splitting starts here

In [ ]:
amazon_fine_food_reviews_dataset['Text']= amazon_fine_food_reviews_dataset['Text'].astype(str)

In [ ]:
#convert y float to int in dataset2
food_reviews_dataset

,Text,y,Source,Restaurant,Unnamed: 4
0,thoroughly enjoyed food and place very good ta...,1.0,Google,Roasters,NaN
1,the ambience of the place was good the menu ha...,1.0,Google,Roasters,NaN
2,loved the food the quality was top notch and t...,1.0,Google,Roasters,NaN
3,great burgers by jessies,1.0,Facebook,Jessies,NaN
4,sauce and mashed potatoes are very less in qua...,0.0,Facebook,Jessies,NaN
...,...,...,...,...,...
56,he view on the drive here was beautiful and we...,0.0,Trip Advisor,NaN,NaN
57,great service provided food quality good as al...,1.0,Trip Advisor,NaN,NaN
58,the most expensive in islamabad with double ta...,0.0,Trip Advisor,NaN,NaN
59,ordered pakistani omelette and chicken sausage...,1.0,Trip Advisor,NaN,NaN


In [ ]:
food_reviews_dataset= food_reviews_dataset.iloc[:61, :]

In [ ]:
food_reviews_dataset['y'] = food_reviews_dataset['y'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
x1 = amazon_fine_food_reviews_dataset.Text
x2 = food_reviews_dataset.Text

In [ ]:
y1 = amazon_fine_food_reviews_dataset.y
y2 = food_reviews_dataset.y

In [ ]:
frame1 = [x1, x2]
X = pd.concat(frame1)
X

0     i have bought several of the vitality canned d...
1     product arrived labeled as jumbo salted peanut...
2     this is a confection that has been around a fe...
3     if you are looking for the secret ingredient i...
4     great taffy at a great price there was a wide ...
                            ...                        
56    he view on the drive here was beautiful and we...
57    great service provided food quality good as al...
58    the most expensive in islamabad with double ta...
59    ordered pakistani omelette and chicken sausage...
60    extremely disappointing service took us over a...
Name: Text, Length: 219553, dtype: object

In [ ]:
frame2 = [y1, y2]
y = pd.concat(frame2)
y = y.values
y

array([1, 0, 1, ..., 0, 1, 0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( 
           X, y, test_size = 0.33, random_state = 700) 

In [ ]:
X_train.shape, y_train.shape

((147100,), (147100,))

In [ ]:
X_test.shape, y_test.shape

((72453,), (72453,))

In [ ]:
tfi = TfidfVectorizer(ngram_range=(1,3), max_features = 2000)

In [ ]:
X_train_vect = tfi.fit_transform(X_train).toarray() 

In [ ]:
X_test_vect = tfi.transform(X_test).toarray()

In [ ]:
#Now model is build and compiled onwards

In [ ]:
logreg = LogisticRegression(max_iter=700)

In [ ]:
logreg.fit(X_train_vect, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=700,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
cv_logreg= logreg.fit(X_train_vect, y_train)

In [ ]:
y_prediction = logreg.predict(X_test_vect)

In [ ]:
from google.colab import files
from sklearn.externals import joblib

# save file in colab environment
joblib.dump(logreg, 'review_classifier_model')

# download to local machine
files.download('review_classifier_model')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Accuracy and prediction scores

In [ ]:
print('Accuracy score: {0:0.2f}'.format(accuracy_score(y_test, y_prediction)))
print('Precision score: {0:0.2f}'.format(precision_score(y_test, y_prediction)))
print('Recall score: {0:0.2f}'.format(recall_score(y_test, y_prediction)))
print('F1 score: {0:0.2f}'.format(f1_score(y_test, y_prediction, average='micro')))

Accuracy score: 0.88
Precision score: 0.90
Recall score: 0.95
F1 score: 0.88


In [ ]:
test_vect = tfi.transform(['''he view on the drive here was beautiful and we managed to get good alfresco seats overlooking the hills. 
They had a good range of items on their menus, from Pakistani dishes to Western food. 
However, the food was less than satisfactory, the taste was just completely off.
 We are not usually complainers, but this was a complete rip-off as the prices were pretty high for this kind of substandard things on the plate which they called food.
Also, we had to witness a family of diners on the next table depositing their garbage into the hills, literally throwing tissues over the fence into the greenery below. 
The staff saw this, did nothing and I was holding down every muscle in my body from picking a fight with these inconsiderate adversaries of the environment.''']).toarray()
logreg.predict(test_vect)

array([0])

In [ ]:
test_vect1 = tfi.transform(['''Great service provided. Food quality good as always. Very busy but the service provided was still top notch. 
Contact free menu's using mobile and sanitisers available all across the restuarant. ''']).toarray()
logreg.predict(test_vect1)

array([1])

In [ ]:
test_vect1 = tfi.transform(['''The most expensive in Islamabad with double taxation ,albeit tax is slashed half by regime.
The french toasts are cooked by putting custard inside and tea/paratha is too expensive.only one cup tea is served in combo deal of paratha.
Other stuff like steak and chinese dishes are too expensive;no fit for pakistan''']).toarray()
logreg.predict(test_vect1)

array([1])

In [ ]:
test_vect1 = tfi.transform(['''Ordered Pakistani omelette and chicken sausage. The food was excellent and prepared perfectly.
It’s a great venue also. I was also surprised by the outstanding service and hygiene. Very good choice and a must go whilst in Islamabad.''']).toarray()
logreg.predict(test_vect1)

array([1])

In [ ]:
test_vect1 = tfi.transform(['''Extremely disappointing service. Took us over an hour to get the food even when there were numerous tables open.
 Never got our coffees or chai even after reminding them three times. Had to even ask to get menus and then the same with water. 
 Only saving grace were the parathas.
Never returning!''']).toarray()
logreg.predict(test_vect1)

array([0])